### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, rek2.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

In [17]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib

Define filepaths. Note the amendments that need to be made to file paths!

In [18]:
pth = r'C:\Users\charl\Documents\T' # change this path to your working folder
fil = r'iceland-latest.osm.pbf' # download this file from geofabrik: http://download.geofabrik.de/europe/iceland.html. 

# be sure to place the .osm.pbf file in the 'tutorial data' folder. 

f = os.path.join(pth, 'tutorial_data', fil)

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [19]:
iceland = losm.OSM_to_network(f)

This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network.

In [20]:
iceland.roads_raw.infra_type.value_counts()

service           25165
footway           15868
track              9953
residential        9519
path               7292
tertiary           4573
unclassified       2997
secondary          2898
primary            2353
trunk              1461
steps               793
trunk_link          283
platform            269
bridleway           247
primary_link        170
cycleway            124
secondary_link       95
living_street        89
tertiary_link        47
motorway             40
motorway_link        36
construction         34
pedestrian           24
proposed             16
road                 14
raceway              12
corridor              8
bus_stop              7
planned               2
rest_area             2
emergency_bay         1
abandoned             1
services              1
Name: infra_type, dtype: int64

We define a list of the types of roads from the above that we consider acceptable for our road network

In [21]:
accepted_road_types = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

We call the filerRoads method and pass it a list of acceptable road types

In [22]:
iceland.filterRoads(acceptedRoads = accepted_road_types)

In [23]:
iceland.roads_raw.infra_type.value_counts()

service           25165
track              9953
residential        9519
tertiary           4573
unclassified       2997
secondary          2898
primary            2353
trunk              1461
trunk_link          283
primary_link        170
secondary_link       95
tertiary_link        47
road                 14
Name: infra_type, dtype: int64

Here, we import a shapefile for our area of interest. I want to make a map of Reykjavik, Iceland. I clip the extent of the country file to just the area around the capital (rek2.shp) below using standard GeoPandas functions:

In [24]:
shp = gpd.read_file(os.path.join(pth, 'tutorial_data', 'rek2.shp'))
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

I check to see everything lines up by running intersect and counting the True / False returns:

In [25]:
iceland.roads_raw.geometry.intersects(shp_obj).value_counts()

False    48689
True     10839
dtype: int64

Then, I do this properly by modifying the roads_raw DF object - I remove any roads that don't intersect the capital area

In [26]:
iceland.roads_raw = iceland.roads_raw.loc[iceland.roads_raw.geometry.intersects(shp_obj) == True]

having narrowed the DataFrame to the roads we are interested in, we generate the RoadsGDF object

In [27]:
iceland.generateRoadsGDF(verbose = False)

With initialReadIn(), we transform this to a graph object

In [28]:
iceland.initialReadIn()

We save this graph object down to file using gn.save()

In [29]:
gn.save(iceland.network,'Iceland_unclean',os.path.join(pth, 'tutorial_outputs'))

Move on to Step 2 to see how we clean up this rough network.